In [1]:
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image
import keras
from tensorflow.keras import layers

2024-03-21 10:34:38.078004: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 10:34:38.487749: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 10:34:39.227465: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate

class MultiOutputModel(Model):
    def __init__(self, input_shape=(224, 224, 6)):
        super(MultiOutputModel, self).__init__()
        self.rgb_branch = self.create_branch(input_shape)
        self.depth_branch = self.create_branch(input_shape)

        # Concatenate the outputs of the two branches
        self.concatenate = concatenate
        
        # Dense layers for combined features
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(64, activation='relu')
        self.dense3 = Dense(5, activation='linear')

    def call(self, inputs):
        rgb_input = inputs[:, :, :, :3]
        depth_input = inputs[:, :, :, 3:]

        # Process each input through its respective branch
        rgb_features = self.rgb_branch(rgb_input)
        depth_features = self.depth_branch(depth_input)

        # Concatenate the outputs of the two branches
        concatenated_features = self.concatenate([rgb_features, depth_features])

        # Further processing
        x = self.dense1(concatenated_features)
        x = self.dense2(x)
        return self.dense3(x)

    def create_branch(self, input_shape):
        """Creates a CNN branch for processing an image."""
        inputs = Input(shape=input_shape)
        x = Conv2D(32, (3, 3), activation='relu')(inputs)
        x = MaxPooling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu')(x)
        x = MaxPooling2D((2, 2))(x)
        x = Flatten()(x)
        return Model(inputs, x)

# Example of compiling and summarizing the model
model = MultiOutputModel()
model.compile(optimizer='adam',
              loss = 'MSE')
model.summary()

2024-03-21 10:34:40.632849: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-21 10:34:40.813770: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "multi_output_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ functional_1 (Functional)       │ ?                      │        20,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_3 (Functional)       │ ?                      │        20,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,512 (158.25 KB)

 Trainable params: 40,512 (158.25 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
import pandas as pd
data = pd.read_pickle('df.pkl')
depths = pd.read_pickle('depths.pkl')
images = pd.read_pickle('images.pkl')


In [4]:
X = np.concatenate([depths, images], axis = -1)
y = np.array(data)[:,1:]

In [5]:
del images, depths, data

In [6]:
X = X.astype(np.float32)
y = y.astype(np.float32)

In [6]:
model.fit(X, y, epochs=10, batch_size=16, validation_split=0.1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).